# LSEG's SFI Snowflake Datawarehouse: Quick Start

In this Quick Start article, we will investigate Public Companies' Climate Data on Snowflake. If you are interested in a more in-depth use-case, please read the "LSEG's SFI Snowflake Datawarehouse: A use-case evaluating public companies' Climate track record using Python and SQL" article that investigates the targets and whether they are on track to meet them; all while using data on LSEG's Sustainable Finance Investing (SFI) Snowflake.

LSEG collects data from a range of sources; for climate data, we rely on our analysts to retrieve the information directly from companies disclosures. The most exciting data onboarded has, recently, related to ESG and Climate topics. From popular demand, we distribute Sustainable Finance and Investing (SFI) data via the Datawarehouse known as [Snowflake](https://docs.snowflake.com/). SFI Snowflake can be accessed via [Snowflake's online platform](https://app.snowflake.com/) or simply using Python via Snowflake's API wrapper named [Snowpark](https://docs.snowflake.com/en/developer-guide/snowpark/python/index). Datasets available include ESG, Climate, EU Taxonomy and Green Revenues data. Snowflake's API is a [REST API](https://blog.postman.com/rest-api-examples/), just like LSEG's Data Platform; similarly, Snowflake created a module to wrap their API into the Python (as well as Scala and Java) module `snowpark` (in their Python library `snowflake`), just as we wrapped LSEG's Data Platform REST API into [LSEG's Data Library](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python) for Python. In this Quick Start, we will look into the Snowflake workflow up to 'Python Worksheets'; in total, there will be three topics: (i) Snowflake's Snowpark Python library, (ii) Snowflake SQL Worksheets and (iii) Snowflake Python Worksheets.

For more information on the data available on LSEG's SFI Snowflake, please look into [myaccount.lseg.com/en/product/quantitative-analytics](https://myaccount.lseg.com/en/product/quantitative-analytics). Here you will find all the related documentation necessary to understand the data-model we are tapping. Please be aware, while reading this documentation, that the SFI datasets are surfaced via the QA Quantitative Analytics (QA) product; LSEG collates several sets of data in tables in QA; some are packaged commercially-speaking under the name of 'Datastream', others under the name 'Worldscope', among these many packages, we are only interested in 'SFI' which is easy to access using Snowflake. When logging into [myaccount.lseg.com/en/product/quantitative-analytics](https://myaccount.lseg.com/en/product/quantitative-analytics), you can access PDF documentation files:

![Image](./Docs/myaccount.lseg.com.quantitative-analytics.png)

Personally, I'm a fan of accessing such information programmatically. In this article, I'll take you through how one can do just that.

## Snowflake's Snowpark Python library

We first need to import the `snowflake-snowpark-python` library. This library allows us to connect to Snowflake's services without having to deal with things like authentication  tokens ourselves:

In [ ]:
## pip install snowflake-snowpark-python
## pip install "snowflake-connector-python[pandas]"
from snowflake import snowpark
import pandas as pd
import os # This is a native library that will allow us to collect authentication  details from the machine where we're running our code.
import plotly.graph_objects as go # `plotly` and its modules are useful for us to plot graphs
import numpy as np # `numpy` in this case will be useful for us to plot graphs

In the cell below, we create the global variable `g_db_session` in which we will insert our authentication  details. This will enable us to invoke it like an object that we can use to access SFI data. To "insert our authentication  details", we will create the `get_session` function:

In [3]:
# Global variable
g_db_session = None

def get_session(user, password):
    global g_db_session
    
    # Establish a connection and assign it to the global session variable
    connection_parameters = { 
        "account": "rddpreprod1.east-us-2.azure",
        "user": user,
        "password": password,
        "warehouse": "SFI_READER_S_WH", 
        "database": "SHAREDB_SFI_MVP",
        "role": "SNO_SNOREFDBD_SFI_READER_ROLE",
        "schema": "DBO",
        "authenticator": "externalbrowser"
    }
    
    g_db_session = snowpark.Session.builder.configs(connection_parameters).create() 

Let's pick an Organisation's Permanent IDentifier (OrgPermID) at random for now:

In [4]:
OrgPermID = "4295875633"

Note that we don't want to show our passwords, or enter it anywhere viewable to anyone reading our code. Therefore, instead of entering our password in a string, we'll create a session in the global `g_db_session` object that opens a browser window where you can enter your credentials and log into Snowflake using your password:

In [5]:
get_session("jonathan.legrand@lseg.com", os.getenv('SSO_PASSWD'))

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/287e9f0e-91ec-4cf0-b7a4-c63898072181/saml2?SAMLRequest=nZJLc9sgFIX%2FioauJQHW2BJjO%2BNEdqtOYrt%2BdZIdlpDDBIEKKIr%2FfZAfnXSRLLpj4Nz7He65w5u3SnivTBuu5AigAAKPyVwVXB5GYLuZ%2BTHwjKWyoEJJNgJHZsDNeGhoJWoyaeyzXLE%2FDTPWc42kId3DCDRaEkUNN0TSihlic7KePNwTHEBCjWHaOhy4lBSGO9aztTUJw7Ztg7YXKH0IMYQwhEnoVJ3kG%2FiAqL9m1FpZlStxLXlzf%2FoEgUIYdQincITlpfCWy%2FMIvqLszyJDfmw2S3%2B5WG%2BAN7n%2B7k5J01RMr5l%2B5Tnbru7PBoxzIAw7%2BOv5YjWdpbfp5Gk73aXTXWCkaktBX1iuqrqxrnXgTmHJilCoA3cDy9IRqF94ke1XBzuYfVdRkaVxO0P9JDE%2FIe418aKZ95a%2Fxa%2FpIzomD49ZDrzdNV7cxZsZ07BMdqFadwVx5CPsw2SDIMERQTDo9%2FET8FIXKpfUnir%2FOu98BBXPtTKqtEoKLtnJJY4HLCkh8xPEcj%2FKS%2BjvBzTy834vTmI4wChGYRcdBuf1IScjevxfQxmGH1tc1nHuEsrSpRI8P3ozpStqPw8QBeh0wwu%2FPEkJqygXk6LQzBgXpBCqvd

Now we can send out SQL query:

In [6]:
# Retrieve the data from Snowflake
query_esg = f"""
    SELECT S.FY, S.SCORECALCDT, S.SCORE, I.TITLE
    FROM ESG2SCORES S 
    JOIN ESG2ITEM I ON S.ITEM = I.ITEM 
    WHERE S.ORGPERMID = {OrgPermID} and I.ITEM IN (2,4,5,6,7,8,9,10,11,12,13,14,15,16,65)
    """

# Execute the query and get the DataFrame
df_esg = g_db_session.sql(query_esg).toPandas()

display(df_esg)

,FY,SCORECALCDT,SCORE,TITLE
0,2008,2022-08-03 10:12:43.353,0.950658,Workforce Score
1,2008,2022-08-03 10:12:43.353,0.900794,Emissions Score
2,2008,2022-08-03 10:12:43.353,0.977876,Resource Use Score
3,2008,2020-12-07 08:49:58.147,0.885417,Management Score
4,2008,2022-08-03 10:12:43.353,0.898116,Social Pillar Score
...,...,...,...,...
303,2017,2024-02-03 06:01:10.670,0.819178,Governance Pillar Score
304,2017,2024-04-06 01:19:13.830,0.526829,ESG Combined Score
305,2017,2024-04-06 01:19:13.830,0.891304,Product Responsibility Score
306,2017,2024-01-06 05:10:37.967,0.898148,CSR Strategy Score


In [7]:
# Pivot the DataFrame to get the desired format
pivot_df_esg = df_esg.pivot(index='FY', columns='TITLE', values='SCORE')

# Return most recent FY
pivot_df_esg = pivot_df_esg.iloc[[-1]].reset_index(drop=True)
pivot_df_esg.columns.name = None

display(pivot_df_esg)

,CSR Strategy Score,Community Score,ESG Combined Score,Emissions Score,Environment Pillar Score,Environmental Innovation Score,Governance Pillar Score,Human Rights Score,Management Score,Product Responsibility Score,Resource Use Score,Shareholders Score,Social Pillar Score,Workforce Score
0,0.95,0.968165,0.481272,0.806513,0.827386,0.761364,0.84,0.909524,0.956,0.787698,0.895062,0.38,0.925768,0.990637


## Snowflake SQL Worksheets

Above, we picked a random OrgPermID, but how can we find the company it relates to? As it turns out, the answer to this question using SFI and SQL code allows us to go quite far.

When you open your Snowflake homepage, you ought to be greeted with something looking like:

![Image](./Docs/SFBrowsr1.PNG)

The main difference should be that you have no ongoing projects.

What we're trying to do here is to find the place where company information lies, then maybe we can find out which company has OrgID '4295875633'. To do this, let's 1st select our Role (for more info on Roles, please read [Snowflake's documentaiton](https://docs.snowflake.com/en/user-guide/security-access-control-overview)):

![Image](./Docs/SFBrowsr2.PNG)

Now that we have selected our role, let's dig through the databases we have access to:

![Image](./Docs/SFBrowsr3.PNG)

In this article, we focus on SFI databases available on Snowflake, specifically "SHAREDB_SFI_MVP" & "SFI_MASTER_MVP":

![Image](./Docs/SFBrowsr4.PNG)

I would always advise starting with the SHAREDB_SFI_MVP.DBO.QASchTableInfo table (i.e.: the SHAREDB_SFI_MVP parent database, the DBO child database, the QASchTableInfo table):

![Image](./Docs/SFBrowsr5.PNG)

![Image](./Docs/SFBrowsr6.PNG)

We can see what data lives in this table with the 'Open in Worksheet' option:

![Image](./Docs/SFBrowsr7.PNG)

We, unfortunately have to pick our role again:

![Image](./Docs/SFBrowsr8.PNG)

I would advise changing the code too, since we don't have permission to change any of the data in tables. I go for:

`SELECT TOP 10 * FROM SHAREDB_SFI_MVP.DBO.QASCHFIELDINFO`

![Image](./Docs/SFBrowsr9.PNG)

## Snowflake Python Worksheets

Let's see what company is linked with this OrgPermID '4295875633'. To do so, we'll need to venture in the world of Python and Snowflake Worksheets. Open a new Python Worksheet:

![Image](./Docs/PythonWrkSht1.png)

Let's run the code we had above, but in Snowflake's Worksheets:

```
# The Snowpark package is required for Python Worksheets. 
# You can add more packages by selecting them using the Packages control and then importing them.
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col
import pandas as pd

def main(session: snowpark.Session): 

    OrgPermID = "4295875633"
    
    query = f"""
    SELECT TABLE_NAME
    FROM SHAREDB_SFI_MVP.INFORMATION_SCHEMA.TABLES
    WHERE TABLE_NAME ILIKE '%Perm%'
    """
    df = session.sql(query).toPandas()
    
    print(df)
```

![Image](./Docs/PythonWrkSht2.png)

- In line 9, `OrgPermID = "4295875633"` precises the company for which we were doing our search.
- Line 11 through to 15 was our SQL query which we will send to Snowflake to run as if we were simply in a SQL Worksheet as before. This code selects the name of all the tables that can be found in the SHAREDB_SFI_MVP database.
- Line 16's `session.sql(query).toPandas()` uses the session created in lines 1 to 8 to send the `query` and transforms it into a pandas dataframe. Pandas dataframes are easy to manipulate in Python, although you can choose other types from which to work. We output this dataframe in the object `df`.
- In line 18, we simply `print` this dataframe so we can see it in the Output window below the code window in Snowflake's browser window.

You may wonder "why did we create python code simply just to send SQL code to Snowflake, when we could do this directly in Snowflake SQL Worksheets?". Great question; the reason is that we can't use loops in SQL Worksheets! Whereas in Python Worksheets, now we can use the table names to search for the ORGPERMID '4295875633' by using the below in the `main` function:

```
    query = f"""
        SELECT TABLE_NAME
        FROM SHAREDB_SFI_MVP.INFORMATION_SCHEMA.TABLES
        WHERE TABLE_NAME ILIKE '%Perm%'
        """
    df = session.sql(query).toPandas()
    
    for table in df.TABLE_NAME:
        try:
            query = f"""
                SELECT * FROM SHAREDB_SFI_MVP.DBO.{table}
                WHERE ORGPERMID = '4295875633'
                """
            _df = session.sql(query).toPandas()
            if len(_df) > 0:
                print("\n")
                print(f"{table}")
                print(_df.T)
        except:
            print("\n")
            print(f"{table} didn't have a 'ORGPERMID' column")
```

![Image](./Docs/PythonWrkSht3.png)

- In line 9 to 13, we create our SQL query, embedding our OrgPermID "4295875633", selecting the table names that were outputted before, all to have this list of tables in the dataframe `df` on line 14.
- In line 16, we start a loop for each table in our `df` and, from line 18 to 22, create & run queries to collect data from these tables.
- Note that, in line 17, we start a try loop. This is due to the fact that some tables will not have an ORGPERMID column, and the code will fail. In line with this, we create an except statement from line 27 to 29.
- Finally, from line 23 to 36, we make sure that if we receive data from our SQL query, we display this data with `print` statements.

We can now see from the `PERMINSTRREF` table that the company probably is "ENI" (as highlighted in red boxes in the image above); but let's check. With this code below, I only output the table and columns names for which "ENI" is found, fully or partially:

```
    query = f"""
        SELECT TABLE_NAME
        FROM SHAREDB_SFI_MVP.INFORMATION_SCHEMA.TABLES
        WHERE TABLE_NAME ILIKE '%Perm%'
        """
    df = session.sql(query).toPandas()
    
    for table in df.TABLE_NAME:
        try:
            query = f"""
                SELECT * FROM SHAREDB_SFI_MVP.DBO.{table}
                WHERE ORGPERMID = '4295875633'
                """
            _df = session.sql(query).toPandas()
            columns_with_name = [col for col in _df.columns if _df[col].astype(str).str.contains('ENI', case=False, na=False).any()]
            if len(_df) > 0 and len(columns_with_name) > 0:
                print("\n")
                print(f"Table {table}'s columns where 'ENI' appears:", columns_with_name)
        except:
            print("\n")
            print(f"{table} didn't have a 'ORGPERMID' column")
```

![Image](./Docs/PythonWrkSht4.png)

I like the sound of 'LEGALNAME'; let's look into it. First let's return to our SQL worksheet:

![Image](./Docs/PythonWrkSht5.png)

and let's use

```
SELECT *
FROM SHAREDB_SFI_MVP.DBO.PERMORGINFO
LIMIT 10;
```

![Image](./Docs/SFBrowsr10.png)

LEGALNAME is indeed quite a lot better; we were right to check all the table and columns names for which "ENI" is found, fully or partially, since it looks like we should focus on table SHAREDB_SFI_MVP.DBO.PERMORGINFO.

While we looked for this OrgId, you saw that data is split in between all the different databases. Looking for the correct database is a little difficult, but once you get a handle of the Python Worksheets, it's easier.

Now, going forward, we are looking for climate data. For this article, I was after Climate data, specifically Green House Gasses Emissions; which is often abbreviated to GHG (i.e.: GHGEmission). Looking into the [myaccount.lseg.com/en/product/quantitative-analytics](https://myaccount.lseg.com/en/product/quantitative-analytics) files related to Climate Data, I found promising information about the SFI_MASTER_MVP.DBO.CLMDataPoint database:

![Image](./Docs/SFBrowsr11.png)

Let's look into SFI_MASTER_MVP.DBO.CLMDataPoint

![Image](./Docs/SFBrowsr12.png)

Here's ORGPERMID again, let's use this. If you look through the documentation as we did to find SFI_MASTER_MVP.DBO.CLMDataPoint, you'll see that there is a table called SFI_MASTER_MVP.DBO.CLMITEM with item names in full; there I found 'ShortTermSet1GHGEmissionPercentageReductionTargeted', which was a good candidate for being the item I'm interested in. We can pick the data in our CLMDataPoint that only relate to this item:

```
SELECT *
FROM SFI_MASTER_MVP.DBO.CLMDataPoint
WHERE ORGPERMID = 4295875633
AND ITEM IN (
    SELECT ITEM
    FROM SFI_MASTER_MVP.DBO.CLMITEM
    WHERE FEEDFIELDNAME = 'ShortTermSet1GHGEmissionPercentageReductionTargeted'
);
```

![Image](./Docs/SFBrowsr13.png)

Let's join our tables to return only the data we're after:

```
SELECT 
    p.ORGPERMID,
    p.LEGALNAME,
    c.FY,
    c.VALUEPRCNT
FROM 
    SHAREDB_SFI_MVP.DBO.PERMORGINFO p
JOIN 
    SFI_MASTER_MVP.DBO.CLMDataPoint c
    ON p.ORGPERMID = c.ORGPERMID
WHERE 
    c.ORGPERMID = 4295875633
    AND c.ITEM IN (
        SELECT ITEM
        FROM SFI_MASTER_MVP.DBO.CLMITEM
        WHERE FEEDFIELDNAME = 'ShortTermSet1GHGEmissionPercentageReductionTargeted'
    );
```

![Image](./Docs/SFBrowsr14.png)

## Conclusion

In this Quick Start, we saw how we can retrieve data from LSEG's SFI Snowflake Datawarehouse. As you could see, there are several ways to do this, via (i) Snowflake's Snowpark Python library, (ii) Snowflake SQL Worksheets and (iii) Snowflake Python Worksheets. One may believe that using the Snowpark Python library is easier than the Snowflake SQL Worksheets, or vice versa; and this really speaks volumes about the versatility of Snowflake, and the number of ways you can retrieve data in the manner that is most efficient for your workflow.

For a longer and more wholesome use-case example, please read the "LSEG's SFI Snowflake Datawarehouse: A use-case evaluating public companies' Climate track record using Python and SQL" article.